In [1]:
import pandas as pd
import numpy as np
import re
import string
from natsort import index_natsorted, order_by_index

In [2]:
!pip install requests
!pip install urllib.parse

ERROR: Could not find a version that satisfies the requirement urllib.parse (from versions: none)
ERROR: No matching distribution found for urllib.parse


In [3]:
def remove_rt(string):
    return re.sub(r"^RT @\w ", "", string)

# From "@musicmadmarc @SocialDilemma_ @netflix @Facebook I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"
# to: "I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"

def remove_beginning_mentions(string):
#     print(string)
#     if(re.search(r'@\w+',string) != None):
    # if(string =='@'):
    new_string=re.sub(r'^[@\w ]*','',string,1)
    new_string = new_string.strip()
    string = remove_beginning_mentions(new_string)
    return string

# From "#musicmadmarc #SocialDilemma_ #netflix #Facebook I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"
# to: "I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"

def remove_beginning_hashs(string):
    # if(string[0]=='#'):
    new_string=re.sub(r'^[#\w ]*','',string,1)
    new_string = new_string.strip()
    string = remove_beginning_mentions(new_string)
    return string

# From " I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K @musicmadmarc @SocialDilemma_ @netflix @Facebook"
# to: "I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"

def remove_last_mentions(string):
    last_mention = string.rfind('@')
    last_space = string.rfind(' ')
    if(last_mention > last_space):
        string = string[:last_space]
        string = remove_last_mentions(string)
    return string

# From " I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K @musicmadmarc @SocialDilemma_ @netflix @Facebook"
# to: "I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"

def remove_last_hashs(string):
    last_mention = string.rfind('#')
    last_space = string.rfind(' ')
    if(last_mention > last_space):
        string = string[:last_space]
        string = remove_last_mentions(string)
    return string

def remove_punctuation_notation(string):
    string = re.sub(r'[\$%&\'()*+\-\/=#@\[\\\]^_`{|}~]*','',string)
    return string

def preprocessing_pipeline(string):
    string = remove_rt(string)
    string = remove_beginning_mentions(string)
    string = remove_beginning_hashs(string)
    string = remove_last_mentions(string)
    string = remove_last_hashs(string)
    string = remove_punctuation_notation(string)
    return string


In [4]:
# mask = pd.read_csv('./output/mask_text.csv')
vaccine = pd.read_csv('./data/vaccine_text.csv')

In [5]:
import requests
from urllib.parse import urlsplit

# Input the t.co format url and return the original link
def to_source_url(shorturl):
    try:
        r = requests.get(shorturl)
        src = urlsplit(r.url)[1]
        return src
    except:
        return None

def find_url(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    return links

In [6]:
text = list(vaccine['text'])

In [7]:
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@','#','RT @']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

tests = [
    "@peter I really love that shirt at #Macy. http://bet.ly//WjdiW4",
    "@shawn Titanic tragedy could have been prevented Economic Times: Telegraph.co.ukTitanic tragedy could have been preve... http://bet.ly/tuN2wx",
    "I am at Starbucks http://4sh.com/samqUI (7419 3rd ave, at 75th, Brooklyn)",
]
for t in tests:
    print(strip_all_entities(strip_links(t)))

I really love that shirt at
Titanic tragedy could have been prevented Economic Times Telegraph co ukTitanic tragedy could have been preve
I am at Starbucks 7419 3rd ave at 75th Brooklyn


In [8]:
text = list(vaccine['text'])
filtered = []
urls = []
for t in text:
    result = find_url(t)
    url_list = []
    for r in result:
        url_list.append(r[0])
    urls.append(url_list)    
    cleared_text = strip_links(t)
    cleared_text = strip_all_entities(cleared_text)
    filtered.append(cleared_text)

In [9]:
vaccine['filtered text'] = filtered

In [10]:
vaccine['urls'] = urls

In [11]:
vaccine = vaccine.reindex(index=order_by_index(vaccine.index, index_natsorted(vaccine['user'])))
vaccine = vaccine.drop_duplicates(subset='                    tweet_id', keep='last')
vaccine = vaccine.reset_index()

In [12]:
users = list(vaccine['user'])

In [43]:
urls = vaccine['urls']

In [15]:
cluster_result = pd.read_csv('./results/pure_accumulated_mention_feature.csv')

In [23]:
set_cluster_0 = set(cluster_result[cluster_result['cluster']==0]['node']) # the general cluster
set_cluster_1 = set(cluster_result[cluster_result['cluster']==1]['node']) # Maybe GOP Cluster
set_cluster_2 = set(cluster_result[cluster_result['cluster']==2]['node']) # Maybe GOP Cluster
# Will Come back to other clusters later

In [35]:
def acquire_cluster(index):
    return set(cluster_result[cluster_result['cluster']==index]['node'])

def fetch_url_statistics(set_cluster):
    set_id_clusterzero = set()
    for i in range(len(users)):
        if users[i] in set_cluster:
            set_id_clusterzero.add(i)
    urls_zero = list()
    for index in set_id_clusterzero:
        urls_zero.append(urls[index])
    urls_zero_fetched = list()
    for lists in urls_zero:
        if lists != []:
            for item in lists:
                urls_zero_fetched.append(item)
    return urls_zero_fetched

In [46]:
url_1_fetched = fetch_url_statistics(set_cluster_1)

In [49]:
def get_src_url(list_url_fecthed):
    print(len(list_url_fecthed))
    src_urls = []
    count = 0
    for urls in list_url_fecthed:
        src_urls.append(to_source_url(urls))
        count += 1
        print(count)
    return src_urls

In [50]:
src_0 = src_urls
src_1 = get_src_url(url_1_fetched)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205


In [51]:
def get_sorted_url(src_list):
    htfreq = [src_list.count(src) for src in src_list]
    htdict = dict(list(zip(src_list, htfreq)))
    sorted_ht = sorted(htdict.items(), key = lambda k: k[1], reverse=True)
    return sorted_ht

In [55]:
get_sorted_url(src_0)

[('twitter.com', 30),
 (None, 7),
 ('edition.cnn.com', 4),
 ('apple.news', 2),
 ('www.fda.gov', 2),
 ('joebiden.com', 1),
 ('www.amazon.com', 1),
 ('www.haaretz.com', 1),
 ('nation.africa', 1),
 ('t.co', 1),
 ('iwillvote.com', 1),
 ('www.survivorcorps.com', 1)]

In [56]:
get_sorted_url(src_1)

[('twitter.com', 98),
 ('edition.cnn.com', 20),
 (None, 13),
 ('www.reuters.com', 10),
 ('www.youtube.com', 8),
 ('www.nytimes.com', 5),
 ('www.arabnews.com', 3),
 ('mobile.twitter.com', 3),
 ('www.msn.com', 3),
 ('principia-scientific.com', 3),
 ('www.ons.gov.uk', 2),
 ('www.fiercebiotech.com', 2),
 ('en.baaghitv.com', 2),
 ('covidactnow.org', 2),
 ('www.independent.co.uk', 1),
 ('www.bloomberg.com', 1),
 ('www.instagram.com', 1),
 ('www.who.int', 1),
 ('news.sky.com', 1),
 ('apnews.com', 1),
 ('time.com', 1),
 ('www.msnbc.com', 1),
 ('www.mayoclinic.org', 1),
 ('www.foreignminister.gov.au', 1),
 ('m.facebook.com', 1),
 ('pakistantimestoday.com', 1),
 ('www.deccanchronicle.com', 1),
 ('www.kiro7.com', 1),
 ('investor.agenusbio.com', 1),
 ('boston.cbslocal.com', 1),
 ('www.channelnewsasia.com', 1),
 ('www.washingtonpost.com', 1),
 ('www.usatoday.com', 1),
 ('www.ft.com', 1),
 ('www.nature.com', 1),
 ('www.theguardian.com', 1),
 ('www.propublica.org', 1),
 ('hbr.org', 1),
 ('www.itamilr

In [57]:
set_cluster_2 = acquire_cluster(2)
url_2_fetched = fetch_url_statistics(set_cluster_2)
src_2 = get_src_url(url_2_fetched)
src_2

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


['twitter.com',
 'www.youtube.com',
 'www.telegraph.co.uk',
 'insiderfinancial.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 '17plus.weebly.com',
 'twitter.com',
 '17plus.weebly.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'edition.cnn.com',
 'www.khaleejtimes.com',
 'www.youtube.com',
 'twitter.com',
 'www.nytimes.com',
 'twitter.com',
 'www.cnbc.com',
 'nypost.com',
 'nypost.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'www.msn.com',
 'www.nytimes.com',
 'www.newsweek.com',
 'twitter.com',
 'www.foxnews.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'linkinghub.elsevier.com',
 'linkinghub.elsevier.com',
 'www.belfasttelegraph.co.uk',
 'twitter.com',
 'twitter.com',
 'mailchi.mp',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'newswrapped.com',
 'newswrapped.com',
 'newswrapped.com',
 'twitter.com',
 'twitter.com',
 'www.businessinside

In [59]:
get_sorted_url(src_2)

[('twitter.com', 34),
 ('www.nytimes.com', 3),
 ('newswrapped.com', 3),
 ('www.youtube.com', 2),
 ('17plus.weebly.com', 2),
 ('nypost.com', 2),
 ('linkinghub.elsevier.com', 2),
 ('www.telegraph.co.uk', 1),
 ('insiderfinancial.com', 1),
 ('edition.cnn.com', 1),
 ('www.khaleejtimes.com', 1),
 ('www.cnbc.com', 1),
 ('www.msn.com', 1),
 ('www.newsweek.com', 1),
 ('www.foxnews.com', 1),
 ('www.belfasttelegraph.co.uk', 1),
 ('mailchi.mp', 1),
 ('www.businessinsider.com', 1),
 ('www.bloomberg.com', 1)]

In [58]:
set_cluster_3 = acquire_cluster(2)
url_3_fetched = fetch_url_statistics(set_cluster_2)
src_3 = get_src_url(url_2_fetched)
src_3

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


['twitter.com',
 'www.youtube.com',
 'www.telegraph.co.uk',
 'insiderfinancial.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 '17plus.weebly.com',
 'twitter.com',
 '17plus.weebly.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'edition.cnn.com',
 'www.khaleejtimes.com',
 'www.youtube.com',
 'twitter.com',
 'www.nytimes.com',
 'twitter.com',
 'www.cnbc.com',
 'nypost.com',
 'nypost.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'www.msn.com',
 'www.nytimes.com',
 'www.newsweek.com',
 'twitter.com',
 'www.foxnews.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'linkinghub.elsevier.com',
 'linkinghub.elsevier.com',
 'www.belfasttelegraph.co.uk',
 'twitter.com',
 'twitter.com',
 'mailchi.mp',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'twitter.com',
 'newswrapped.com',
 'newswrapped.com',
 'newswrapped.com',
 'twitter.com',
 'twitter.com',
 'www.businessinside

In [60]:
get_sorted_url(src_3)

[('twitter.com', 34),
 ('www.nytimes.com', 3),
 ('newswrapped.com', 3),
 ('www.youtube.com', 2),
 ('17plus.weebly.com', 2),
 ('nypost.com', 2),
 ('linkinghub.elsevier.com', 2),
 ('www.telegraph.co.uk', 1),
 ('insiderfinancial.com', 1),
 ('edition.cnn.com', 1),
 ('www.khaleejtimes.com', 1),
 ('www.cnbc.com', 1),
 ('www.msn.com', 1),
 ('www.newsweek.com', 1),
 ('www.foxnews.com', 1),
 ('www.belfasttelegraph.co.uk', 1),
 ('mailchi.mp', 1),
 ('www.businessinsider.com', 1),
 ('www.bloomberg.com', 1)]